In [64]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
raw_2019 = pd.read_csv("./survey_results_public_2019.csv")

In [4]:
raw_2019.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource',
       'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor',
       'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode',
       'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney',
       'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz',
       'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc',
       'CompTotal', 'CompFreq', 'ConvertedComp', 'WorkWeekHrs', 'WorkPlan',
       'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev',
       'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat',
       'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife'

In [107]:
raw_2019['DevType'].head(20)

0                                                   NaN
1     Developer, desktop or enterprise applications;...
2     Designer;Developer, back-end;Developer, front-...
3                                 Developer, full-stack
4     Academic researcher;Developer, desktop or ente...
5     Data or business analyst;Data scientist or mac...
6                         Designer;Developer, front-end
7        Developer, back-end;Engineer, site reliability
8     Database administrator;Developer, back-end;Dev...
9     Data or business analyst;Data scientist or mac...
10                                                  NaN
11    Data or business analyst;Data scientist or mac...
12    Data or business analyst;Database administrato...
13        Developer, desktop or enterprise applications
14                                              Student
15                                Developer, full-stack
16    Developer, back-end;Developer, front-end;Devel...
17                             Data or business 

In [15]:
languages = raw_2019[['Respondent', 'LanguageWorkedWith']]

In [16]:
b = languages['LanguageWorkedWith'].str.split(';', expand=True)

In [29]:
l = []
for i in range(28):
    for j in b[i].unique():
        l.append(j)

In [32]:
all_lang = list(dict.fromkeys(l))

In [117]:
df_2019 = raw_2019[['Respondent', 'Employment', 'EdLevel', 'UndergradMajor', 'OrgSize','DevType', 'YearsCode', 'YearsCodePro', 'ConvertedComp', 'Age', 'Gender', 'Country']]

In [35]:
df_2019.head()

,Respondent,Employment,EdLevel,UndergradMajor,OrgSize,DevType,YearsCode,ConvertedComp,Age,Gender,Country
0,1,"Not employed, and not looking for work",Primary/elementary school,NaN,NaN,NaN,4,NaN,14.0,Man,United Kingdom
1,2,"Not employed, but looking for work","Secondary school (e.g. American high school, G...",NaN,NaN,"Developer, desktop or enterprise applications;...",NaN,NaN,19.0,Man,Bosnia and Herzegovina
2,3,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,100 to 499 employees,"Designer;Developer, back-end;Developer, front-...",3,8820.0,28.0,Man,Thailand
3,4,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",100 to 499 employees,"Developer, full-stack",3,61000.0,22.0,Man,United States
4,5,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...","10,000 or more employees","Academic researcher;Developer, desktop or ente...",16,NaN,30.0,Man,Ukraine


In [36]:
df_2019.shape

(88883, 11)

In [118]:
df_2019 = df_2019[~df_2019['ConvertedComp'].isna()]

In [119]:
avg_by_country = df_2019.groupby(by=['Country'])['ConvertedComp'].mean()

In [120]:
df_2019 = df_2019.merge(avg_by_country, left_on='Country', right_index=True, how='left').rename(columns={'ConvertedComp_x':'ConvertedComp', 'ConvertedComp_y':'CountryAvgComp'})

In [121]:
avg_by_ed_lvl = df_2019.groupby(by=['EdLevel'])['ConvertedComp'].mean()

In [122]:
df_2019 = df_2019.merge(avg_by_ed_lvl, left_on='EdLevel', right_index=True, how='left').rename(columns={'ConvertedComp_x':'ConvertedComp', 'ConvertedComp_y':'EducationAvgComp'})

In [123]:
avg_by_major = df_2019.groupby(by=['UndergradMajor'])['ConvertedComp'].mean()

In [124]:
df_2019 = df_2019.merge(avg_by_major, left_on='UndergradMajor', right_index=True, how='left').rename(columns={'ConvertedComp_x':'ConvertedComp', 'ConvertedComp_y':'MajorAvgComp'})

In [63]:
df_2019.groupby(by=['Gender']).count()

,Respondent,Employment,EdLevel,UndergradMajor,OrgSize,DevType,YearsCode,ConvertedComp,Age,Country,CountryAvgComp,EducationAvgComp,MajorAvgComp
Gender,,,,,,,,,,,,,
Man,50503,50387,49787,46367,49671,49872,50441,50503,48735,50503,50503,49787,46367
"Man;Non-binary, genderqueer, or gender non-conforming",115,115,111,112,113,114,115,115,113,115,115,111,112
"Non-binary, genderqueer, or gender non-conforming",342,341,334,300,333,335,339,342,318,342,342,334,300
Woman,3762,3754,3701,3500,3660,3703,3759,3762,3642,3762,3762,3701,3500
Woman;Man,40,40,39,35,40,39,40,40,35,40,40,39,35
"Woman;Man;Non-binary, genderqueer, or gender non-conforming",25,24,22,16,24,24,22,25,22,25,25,22,16
"Woman;Non-binary, genderqueer, or gender non-conforming",106,106,106,88,103,105,104,106,105,106,106,106,88


In [125]:
df_2019['Gender'] = df_2019['Gender'].fillna('Not Specified')

In [126]:
le = preprocessing.LabelEncoder()
le.fit(list(df_2019['Gender'].unique()))

LabelEncoder()

In [127]:
df_2019['GenderCode'] = le.transform(df_2019['Gender'])

In [128]:
df_2019.loc[df_2019['OrgSize']=='1,000 to 4,999 employees', 'NewOrgSize'] = 3000
df_2019.loc[df_2019['OrgSize']=='10 to 19 employees', 'NewOrgSize'] = 15
df_2019.loc[df_2019['OrgSize']=='10,000 or more employees', 'NewOrgSize'] = 10000
df_2019.loc[df_2019['OrgSize']=='100 to 499 employees', 'NewOrgSize'] = 300
df_2019.loc[df_2019['OrgSize']=='2-9 employees', 'NewOrgSize'] = 5
df_2019.loc[df_2019['OrgSize']=='20 to 99 employees', 'NewOrgSize'] = 60
df_2019.loc[df_2019['OrgSize']=='5,000 to 9,999 employees', 'NewOrgSize'] = 7500
df_2019.loc[df_2019['OrgSize']=='500 to 999 employees', 'NewOrgSize'] = 750
df_2019.loc[df_2019['OrgSize']=='Just me - I am a freelancer, sole proprietor, etc.', 'NewOrgSize'] = 1

In [129]:
orgSizeMean = df_2019['NewOrgSize'].mean()

In [130]:
df_2019['NewOrgSize'] = df_2019['NewOrgSize'].fillna(value=orgSizeMean)

In [134]:
avgComp = df_2019['ConvertedComp'].mean()

In [135]:
df_2019['EducationAvgComp'] = df_2019['EducationAvgComp'].fillna(value=avgComp)

In [137]:
df_2019['MajorAvgComp'] = df_2019['MajorAvgComp'].fillna(value=avgComp)

In [145]:
df_2019 = df_2019.reset_index().drop('index', axis=1)

In [149]:
df_2019['DevType'].str.split(';', expand=True)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,Designer,"Developer, back-end","Developer, front-end","Developer, full-stack",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"Developer, full-stack",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Engineer, data",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Database administrator,"Developer, back-end","Developer, front-end","Developer, full-stack","Developer, QA or test",DevOps specialist,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Data or business analyst,Data scientist or machine learning specialist,Database administrator,"Developer, back-end","Developer, desktop or enterprise applications","Developer, front-end","Developer, full-stack","Developer, game or graphics",Educator,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55818,"Developer, back-end","Developer, front-end","Developer, full-stack",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
55819,"Developer, desktop or enterprise applications","Developer, embedded applications or devices","Developer, game or graphics","Developer, QA or test",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
55820,"Developer, full-stack","Engineer, site reliability",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
55821,Academic researcher,Database administrator,"Developer, back-end","Developer, desktop or enterprise applications","Developer, embedded applications or devices","Developer, front-end","Developer, full-stack","Developer, game or graphics","Developer, mobile",Educator,...,None,None,None,None,None,None,None,None,None,None


In [147]:
for i in range(df_N)df.iloc[1].as_matrix()

,0,1,2,3,4,5,6,7,8,9,...,55813,55814,55815,55816,55817,55818,55819,55820,55821,55822
Respondent,3,4,6,9,10,13,14,16,17,18,...,88871,88873,88874,88876,88877,88878,88879,88881,88882,88883
Employment,Employed full-time,Employed full-time,Employed full-time,Employed full-time,Employed full-time,Employed full-time,Employed full-time,Employed full-time,Employed full-time,Employed full-time,...,Employed full-time,"Independent contractor, freelancer, or self-em...",Employed full-time,Employed full-time,Employed full-time,Employed full-time,Employed full-time,Employed full-time,Employed full-time,Employed full-time
EdLevel,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Bachelor’s degree (BA, BS, B.Eng., etc.)","Bachelor’s degree (BA, BS, B.Eng., etc.)",Some college/university study without earning ...,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Master’s degree (MA, MS, M.Eng., MBA, etc.)","Other doctoral degree (Ph.D, Ed.D., etc.)","Master’s degree (MA, MS, M.Eng., MBA, etc.)","Bachelor’s degree (BA, BS, B.Eng., etc.)","Master’s degree (MA, MS, M.Eng., MBA, etc.)",...,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Some college/university study without earning ...,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Bachelor’s degree (BA, BS, B.Eng., etc.)","Bachelor’s degree (BA, BS, B.Eng., etc.)","Bachelor’s degree (BA, BS, B.Eng., etc.)","Master’s degree (MA, MS, M.Eng., MBA, etc.)","Master’s degree (MA, MS, M.Eng., MBA, etc.)","Master’s degree (MA, MS, M.Eng., MBA, etc.)","Bachelor’s degree (BA, BS, B.Eng., etc.)"
UndergradMajor,Web development or web design,"Computer science, computer engineering, or sof...",Mathematics or statistics,"Computer science, computer engineering, or sof...",NaN,"Computer science, computer engineering, or sof...","Computer science, computer engineering, or sof...",NaN,"Computer science, computer engineering, or sof...","Computer science, computer engineering, or sof...",...,"Information systems, information technology, o...","Computer science, computer engineering, or sof...","A social science (ex. anthropology, psychology...","Computer science, computer engineering, or sof...","Computer science, computer engineering, or sof...","Computer science, computer engineering, or sof...","Computer science, computer engineering, or sof...","Computer science, computer engineering, or sof...","Computer science, computer engineering, or sof...","Computer science, computer engineering, or sof..."
OrgSize,100 to 499 employees,100 to 499 employees,NaN,10 to 19 employees,"10,000 or more employees",10 to 19 employees,100 to 499 employees,100 to 499 employees,20 to 99 employees,"1,000 to 4,999 employees",...,20 to 99 employees,"Just me - I am a freelancer, sole proprietor, ...","10,000 or more employees","10,000 or more employees",500 to 999 employees,20 to 99 employees,20 to 99 employees,100 to 499 employees,100 to 499 employees,20 to 99 employees
DevType,"Designer;Developer, back-end;Developer, front-...","Developer, full-stack",Data or business analyst;Data scientist or mac...,"Database administrator;Developer, back-end;Dev...",Data or business analyst;Data scientist or mac...,Data or business analyst;Database administrato...,"Developer, desktop or enterprise applications","Developer, full-stack","Developer, back-end;Developer, front-end;Devel...",Data or business analyst,...,"Developer, back-end;Developer, embedded applic...",Data scientist or machine learning specialist;...,Data scientist or machine learning specialist;...,"Developer, back-end;Developer, game or graphics",Data scientist or machine learning specialist;...,"Developer, back-end;Developer, front-end;Devel...","Developer, desktop or enterprise applications;...","Developer, full-stack;Engineer, site reliability",Academic researcher;Database administrator;Dev...,"Developer, back-end;Developer, desktop or ente..."
YearsCode,3,3,13,12,12,17,13,10,5,10,...,5,27,7,8,31,12,17,18,10,9
YearsCodePro,1,Less than 1 year,3,4,10,8,2,3,2,3,...,2,21,4,2,28,3,7,9,8,5
ConvertedComp,882